In [2]:
%run useful_functions.py

C:\Users\natha\Desktop\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\Work_Nathan\useful_functions.py:951: SyntaxWarning: invalid escape sequence '\s'
  item1A = re.sub('\s{2,}',' ', item1A)
C:\Users\natha\Desktop\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\Work_Nathan\useful_functions.py:962: SyntaxWarning: invalid escape sequence '\s'
  document = re.sub('\s{2,}',' ', document)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


update 21


In [3]:
import wrds

In [4]:
#you can skip the following section

## Check for discontinuities

In [5]:
#I first want to check if there are discontinuities through the years for the 10k

#create a list of all available firms
list_available_firms=[]
years=list(range(2007, 2025))
for year in years:
        
        path_ref='../../data/10k_statements_new/tokens/'+str(year)
    
        files = glob.glob(path_ref + '/*')

        #print(len(files) == len(set(files)))
        
        list_available_firms+= [file.split('\\')[-1].split('_')[0] for file in files]
    
list_available_firms=list(set(list_available_firms))


In [6]:
def file_appears_in_directory(filename, directory):
    full_path = os.path.join(directory, filename)
    return os.path.exists(full_path) and os.path.isfile(full_path)

In [7]:
# Example list of firm names
files_name = [f"{name}_tokens.csv.gz" for name in list_available_firms]

# Example directory where files are stored
directory = '../../data/10k_statements_new/tokens/'


data_list=[list_available_firms]
for year in years:
    list_of_presence=[]
    for file in files_name:
        if file_appears_in_directory(file,directory+str(year)+"/"):
            list_of_presence.append(1)
        else:
            list_of_presence.append(0)
    data_list.append(list_of_presence)



# Create a list of values ranging from 2007 to 2024
list_columns = years

# Prepend "firms" to the beginning of the list
list_columns.insert(0, "firms")
df = pd.DataFrame(list(map(list, zip(*data_list))), columns=list_columns)

In [8]:
df.sample(n=10, axis=0)

,firms,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
3305,KNXA,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
5653,APAGF,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
5132,TEKK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
2900,AVNT,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5714,TNGO,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0
6512,CVLY,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
6494,TOI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0
4117,SIG,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
4529,WEAV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
751,HIW,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [9]:
def has_discontinuity(row):
    discontinuity=False
    started_seq_ones=False
    finished_seq_ones=False
    for bit in row:
        if bit==1 and not(started_seq_ones):
            started_seq_ones=True

        elif bit==0 and started_seq_ones:
            finished_seq_ones=True

        elif bit==1 and finished_seq_ones:
            discontinuity=True

    return discontinuity


            
        

In [10]:
df['has_discontinuity'] = df[years].apply(has_discontinuity, axis=1)

In [11]:
df

,firms,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,has_discontinuity
0,MCEL,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
1,OSH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,False
2,CACB,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,False
3,WE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,False
4,XOS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7078,TRMA,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
7079,UMH,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,False
7080,WHG,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,False
7081,NN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,False


# Extracting the data and saved them

In [12]:
# Idea: we download and create for each stock a dataframe containing the data, 
# then, at the end, we put them all together to have a nice dataframe for each firms

#read in the previously saved stocknames file
stocknames = pd.read_csv("../../data/stocknames.csv.gz", na_filter = False)
stocknames.replace('', np.nan, inplace = True)

In [13]:
stocknames.columns

Index(['CIK', 'comnam', 'end_date', 'filling_date_2007', 'filling_date_2008',
       'filling_date_2009', 'filling_date_2010', 'filling_date_2011',
       'filling_date_2012', 'filling_date_2013', 'filling_date_2014',
       'filling_date_2015', 'filling_date_2016', 'filling_date_2017',
       'filling_date_2018', 'filling_date_2019', 'filling_date_2020',
       'filling_date_2021', 'filling_date_2022', 'filling_date_2023',
       'filling_date_2024', 'permco', 'st_date', 'ticker', 'url_2007',
       'url_2008', 'url_2009', 'url_2010', 'url_2011', 'url_2012', 'url_2013',
       'url_2014', 'url_2015', 'url_2016', 'url_2017', 'url_2018', 'url_2019',
       'url_2020', 'url_2021', 'url_2022', 'url_2023', 'url_2024'],
      dtype='object')

In [14]:
stocknames["ticker"].is_unique #sanity check

True

In [15]:
stocknames["ticker"]

0        TSLA
1        HBIO
2       BELFA
3        AMPX
4        RENT
        ...  
7078     GWIN
7079     CLCN
7080     BRVO
7081     BRER
7082     ATDS
Name: ticker, Length: 7083, dtype: object

In [16]:
#create a empty file for each ticker
directory_fin_var="Financial_values/"

for name in stocknames["ticker"].values:
    file_path = os.path.join(directory_fin_var, name)
    if not os.path.exists(file_path):
            os.makedirs(file_path)
            pass

In [17]:
#connect to WRDS
db = wrds.Connection(wrds_username = 'nmonnet')
db.create_pgpass_file()

Loading library list...
Done


Enter your WRDS username [nmonnet]: nmonnet
Enter your password: ········


### Monthly Returns, Industry Codes, Market Capitalisation, Volumes

In [18]:
SIC_to_FF = pd.read_excel('../../data/SIC_to_FF_industry.xlsx', skiprows = 3).dropna(axis = 1, how = 'all')

volumes_list=[]
market_list=[]

for i in tqdm(range(len(stocknames)),desc = 'Getting monthly returns, volumes, market caps and industry class for all firms', leave = False):
   
        no_error=True
        for nb_attempt in range(10):
            try:


                permco=stocknames.permco[i]
                start_date=stocknames.st_date[i]
                end_date=stocknames.end_date[i]
                Request = """select date,  ret, shrout, prc, vol, hsiccd from crsp.msf where permco in ({})
                          and date >= '{}' and date <='{}'""".format(permco, start_date, end_date)
                data = db.raw_sql(Request)
                data.set_index('date', inplace = True)
                data.index = pd.to_datetime(data.index)
                data.index = data.index.to_period('M')
                
                #there are months with several observations for some reason
                data = data[~data.index.duplicated(keep='first')]
                
                #calculate market cap
                data['Market Cap'] = data['shrout']*1000*data['prc'].abs() #shrout in thousands
                data_market=data['Market Cap'].copy()
                

                
                data_returns = data["ret"].copy()

                # Drop returns that are missing (==-66, -77, -88, or -99)
                missing_returns = [-66, -77, -88, -99]
                data_returns = data_returns[~np.isin(data_returns, missing_returns)]


                data_volumes=data['vol'].copy()
                
                missing_volumes = [ -99]
                data_volumes = data_volumes[~np.isin(data_volumes, missing_volumes)]*100*data['prc'][~np.isin(data_volumes, missing_volumes)]
            
                
                #replace missing values from the industry classifications with NaN
                data_industry=data[['hsiccd']].copy()
                data_industry[data_industry.hsiccd == 0] = np.nan

                data_industry.rename(columns={'hsiccd': 'industry_codes'}, inplace=True)

                
                
                data_industry['FF_Industry_codes'] = data_industry.industry_codes.apply(lambda x: match_FF_industry(x,SIC_to_FF))
                
                
                # Specify the directory path where you want to save the file
                directory_path = 'Financial_values/'+stocknames.ticker[i]+"/"
                
                #returns.to_csv(directory_path+'monthly_returns.csv.gz', compression='gzip')
                #industry_codes.to_csv(directory_path+'industry_codes.csv.gz', compression='gzip')
                #market_caps.to_csv(directory_path+'market_caps.csv.gz', compression='gzip')
            
                data_returns.to_csv(directory_path+'monthly_returns.csv')
                data_industry.to_csv(directory_path+'industry_codes.csv')
                data_market.to_csv(directory_path+'market_caps.csv')


                data_market.rename(stocknames.ticker[i], inplace=True)
                data_volumes.rename(stocknames.ticker[i], inplace=True)
                market_list.append(data_market)
                volumes_list.append(data_volumes)

                
    
            except:
                no_error=False
                
            if no_error:
                break
        if not(no_error):
            print("There was a problem that could not be resolved in 10 tries")
            print(stocknames.ticker[i])
            print(i)

    
        #break #to be removed

Getting monthly returns, volumes, market caps and industry class for all firms:   0%|          | 0/7083 [00:00…

There was a problem that could not be resolved in 10 tries
RAND
4020
There was a problem that could not be resolved in 10 tries
UTI
4021
There was a problem that could not be resolved in 10 tries
TRIP
4022
There was a problem that could not be resolved in 10 tries
TPX
4023
There was a problem that could not be resolved in 10 tries
PIPR
4024
There was a problem that could not be resolved in 10 tries
NOC
4025
There was a problem that could not be resolved in 10 tries
COO
4026
There was a problem that could not be resolved in 10 tries
NG
4027
There was a problem that could not be resolved in 10 tries
TDC
4028
There was a problem that could not be resolved in 10 tries
FBIO
4029
There was a problem that could not be resolved in 10 tries
USLM
4030
There was a problem that could not be resolved in 10 tries
KAMN
4031
There was a problem that could not be resolved in 10 tries
APLD
4032
There was a problem that could not be resolved in 10 tries
CPRI
4033
There was a problem that could not be res


KeyboardInterrupt



In [ ]:
market_caps_df = pd.concat(market_list, axis = 1).sort_index()
market_caps_df = market_caps_df[market_caps_df.index.year > 2006]

volumes_df = pd.concat(volumes_list, axis = 1).sort_index()
volumes_df = volumes_df[volumes_df.index.year > 2006]
          

In [56]:
display(market_caps_df)
save_path_ratio='Financial_ratios/'
market_caps_df.to_csv(save_path_ratio + 'market_caps.csv.gz')

,TSLA,HBIO,BELFA,AMPX,RENT,INLX,NTRS,PYXS,SPGI,XPER,...,FLYD,BCRD,MSPC,MSTC,HRST,GWIN,CLCN,BRVO,BRER,ATDS
date,,,,,,,,,,,,,,,,,,,,,
2007-01,NaN,161978600.0,75684000.0,NaN,NaN,NaN,1.328609e+10,NaN,2.375974e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,NaN,154979760.0,77711250.0,NaN,NaN,NaN,1.322264e+10,NaN,2.291467e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,NaN,147352220.0,102119340.0,NaN,NaN,NaN,1.319514e+10,NaN,2.209188e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,NaN,169363340.0,94037370.0,NaN,NaN,NaN,1.381167e+10,NaN,2.248990e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,NaN,158077920.0,102117120.0,NaN,NaN,NaN,1.427901e+10,NaN,2.413039e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08,8.191443e+11,188254080.0,108620820.0,424473000.0,90199560.0,15033060.0,1.574679e+10,95640960.0,1.243716e+11,510904600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09,7.954494e+11,183558400.0,100888200.0,409725600.0,44823635.4,19005210.0,1.438486e+10,88145060.0,1.162735e+11,426908420.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10,6.384545e+11,187400320.0,110891340.0,251540400.0,35731768.8,17355240.0,1.364574e+10,73970980.0,1.111504e+11,367591530.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
display(volumes_df)
save_path_ratio='Financial_ratios/'
volumes_df.to_csv(save_path_ratio + 'volumes.csv.gz')

In [58]:
volume_per_cap = volumes_df.div(market_caps_df)

In [59]:
display(volume_per_cap)
save_path_ratio='Financial_ratios/'
volume_per_cap.to_csv(save_path_ratio + 'volume_per_cap.csv.gz')

,TSLA,HBIO,BELFA,AMPX,RENT,INLX,NTRS,PYXS,SPGI,XPER,...,FLYD,BCRD,MSPC,MSTC,HRST,GWIN,CLCN,BRVO,BRER,ATDS
date,,,,,,,,,,,,,,,,,,,,,
2007-01,NaN,0.041264,0.045986,NaN,NaN,NaN,0.130842,NaN,0.094688,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,NaN,0.029551,0.050647,NaN,NaN,NaN,0.068157,NaN,0.099106,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,NaN,0.053803,0.221162,NaN,NaN,NaN,0.106598,NaN,0.159161,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,NaN,0.048023,0.147836,NaN,NaN,NaN,0.095803,NaN,0.110272,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,NaN,0.067654,0.093341,NaN,NaN,NaN,0.102803,NaN,0.120589,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08,0.788570,0.081714,0.038375,0.051809,0.195393,0.036181,0.096797,0.081291,0.073773,0.083833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09,0.767394,0.055655,-0.051401,0.204946,0.428919,-0.047275,0.116781,0.085136,0.072078,0.088380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10,0.814921,0.042073,0.045191,0.088859,0.155857,0.022263,0.194769,0.042272,0.074194,0.120218,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Daily prices, open/close/high/low

In [10]:
for i in tqdm(range(len(stocknames)),desc = 'Getting daily prices o/c/h/l for all firms', leave = False):
    no_error=True
    for nb_attempt in range(10):
        try:
            permco=stocknames.permco[i]
            start_date=subtract_one_month(stocknames.st_date[i])
            end_date=stocknames.end_date[i]
            Request = """select date, prc, openprc, bidlo, askhi, cfacpr from crsp.dsf where permco in ({})
                          and date >= '{}' and date <='{}'""".format(permco, start_date, end_date)
            data = db.raw_sql(Request)
        
            data.set_index('date', inplace = True)
            data.index = pd.to_datetime(data.index)
            price_table=data.sort_index()
        
            if (price_table['openprc'] <= 0).any() or (price_table['bidlo'] <= 0).any() or (price_table['askhi'] <= 0).any():
                raise ValueError("Contains zero or negative value where they should not appear")
            
            # Replace None values with np.nan
            price_table = price_table.replace('None', np.nan)
            
            # Replace zero and negative values in 'prc' column with np.nan
            price_table['prc'] = price_table['prc'].apply(lambda x: np.nan if x <= 0 else x)
            
            # For each row where 'prc' is nan, make the rest of the row also contain only nan
            price_table.loc[price_table['prc'].isnull(), :] = np.nan
        
            
            
            price_table=price_table.dropna()
        
            
            #Finally, we drop a few observations where the open or close prices are outside the high-low
            #range or where the low price is higher than the high price. -ardiaGuidottiKroencke2023W
        
            price_table = price_table[(price_table['prc'] <= price_table['askhi']) &
                                  (price_table['prc'] >= price_table['bidlo']) &
                                  (price_table['openprc'] <= price_table['askhi']) &
                                  (price_table['openprc'] >= price_table['bidlo'])]
        
            directory_path = 'Financial_values/'+stocknames.ticker[i]+"/"
            price_table.to_csv(directory_path+'daily_prices.csv')

        except:
            no_error=False
            
        if no_error:
            break
    if not(no_error):
        print("There was a problem that could not be resolved in 10 tries")
        print(i)
    #break #to be removed
 

Getting daily prices o/c/h/l for all firms:   0%|          | 0/7083 [00:00<?, ?it/s]

### Monthly prices, monthly returns without dividends

In [10]:
for i in tqdm(range(len(stocknames)),desc = 'Getting monthly prices and monthly returns without dividends for all firms', leave = False):

    no_error=True
    for nb_attempt in range(10):
        try:
            permco=stocknames.permco[i]
            start_date=stocknames.st_date[i]
            end_date=stocknames.end_date[i]
            Request = """select date, retx, prc, cfacpr from crsp.msf where permco in ({})
                          and date >= '{}' and date <='{}'""".format(permco, start_date, end_date)
            data = db.raw_sql(Request)
            
            data.set_index('date', inplace = True)
            data.index = pd.to_datetime(data.index)
            data.index = data.index.to_period('M')
            #there are months with several observations for some reason
            data = data[~data.index.duplicated(keep='first')]
            
            data['prc']=data['prc'].abs()
            
            data.loc[data['prc'] == 0, 'prc'] = np.nan
            
            data['retx']=data['retx'].replace([-66,-77,-88,-99], np.nan)
            
            data=data.dropna() # Drop rows containing at least one NaN value
            
            data[['prc']]
        
            directory_path = 'Financial_values/'+stocknames.ticker[i]+"/"
            data[['prc']].to_csv(directory_path+'monthly_prices.csv')
            data[['retx']].to_csv(directory_path+'monthly_returns_no_div.csv')
            data[['cfacpr']].to_csv(directory_path+'monthly_cfacpr.csv')

        except:
            no_error=False
            
        if no_error:
            break
    if not(no_error):
        print("There was a problem that could not be resolved in 10 tries")
        print(i)
    #break #to be removed


Getting monthly prices and monthly returns without dividends for all firms:   0%|          | 0/7083 [00:00<?, …

### Other financial ratio

In [23]:
#########################download yearly book values for all companies
book_value = db.get_table(library = 'comp', table = 'funda', columns = ['datadate', 'tic', 'ceq', 'gvkey'])
book_value.columns = ['date', 'ticker','BV', 'gvkey']
book_value = book_value[book_value.ticker.isin(stocknames.ticker)].drop_duplicates(subset = ['ticker','date'])

temp = book_value[['ticker', 'gvkey']]
temp = temp.groupby('ticker').last()
book_value.drop(columns = ['gvkey'], inplace = True)
stocknames = stocknames.merge(temp, on = 'ticker', how = 'left')

book_value.dropna(how = 'any', inplace = True)
book_value.date = pd.to_datetime(book_value.date)
book_value = book_value.pivot(index = 'date', columns = 'ticker', values = 'BV')

book_value=book_value.astype(float)

#Winsorize by year
book_value = book_value.groupby(book_value.index.year).apply(lambda x: x.stack().clip(lower = x.stack().quantile(0.01), upper = x.stack().quantile(0.99)).unstack())

book_value = book_value.ffill()

book_value=book_value.reset_index(level=0, drop=True)

book_value.index = book_value.index.to_period('M')
book_value = book_value.multiply(1000000) #in millions on wrds
temp_book_value = book_value[book_value.index.year < 2025] ### important for later!
book_value = book_value[(book_value.index.year > 2006) & (book_value.index.year < 2025)]
book_value[book_value<0] = np.nan
book_value.dropna(how = 'all', axis = 1, inplace = True)

In [24]:
display(book_value)
save_path_ratio='Financial_ratios/'
book_value.to_csv(save_path_ratio + 'book_values.csv.gz')

ticker,BAC,CMA,NCC,PNC,WFC,JPM,CP,ENB,IMO,OKE,...,SOUN,TMC,VIRX,WALD,WEST,WULF,ZFOX,ME,ROIV,BRCC
date,,,,,,,,,,,,,,,,,,,,,
2007-01,2.403812e+10,5.153000e+09,1.458100e+10,1.078800e+10,2.403812e+10,2.403812e+10,4.856500e+09,4.485600e+09,7.406000e+09,2.215958e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,2.403812e+10,5.153000e+09,1.458100e+10,1.078800e+10,2.403812e+10,2.403812e+10,4.856500e+09,4.485600e+09,7.406000e+09,2.215958e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,2.403812e+10,5.153000e+09,1.458100e+10,1.078800e+10,2.403812e+10,2.403812e+10,4.856500e+09,4.485600e+09,7.406000e+09,2.215958e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,2.403812e+10,5.153000e+09,1.458100e+10,1.078800e+10,2.403812e+10,2.403812e+10,4.856500e+09,4.485600e+09,7.406000e+09,2.215958e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,2.403812e+10,5.153000e+09,1.458100e+10,1.078800e+10,2.403812e+10,2.403812e+10,4.856500e+09,4.485600e+09,7.406000e+09,2.215958e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,4.596862e+10,4.787000e+09,1.340783e+10,4.002800e+10,4.596862e+10,4.596862e+10,3.888600e+10,4.596862e+10,2.241300e+10,6.493885e+09,...,NaN,41505000.0,55651000.0,639237000.0,9269000.0,108481000.0,488871000.0,713939000.0,1.157766e+09,NaN
2023-11,4.596862e+10,4.787000e+09,1.340783e+10,4.002800e+10,4.596862e+10,4.596862e+10,3.888600e+10,4.596862e+10,2.241300e+10,6.493885e+09,...,NaN,41505000.0,55651000.0,639237000.0,9269000.0,108481000.0,488871000.0,713939000.0,1.157766e+09,NaN
2023-12,5.628672e+10,4.787000e+09,1.340783e+10,4.002800e+10,5.628672e+10,5.628672e+10,4.149200e+10,5.463600e+10,2.222200e+10,1.648400e+10,...,13981000.0,41505000.0,55651000.0,639237000.0,113740000.0,108481000.0,488871000.0,713939000.0,1.157766e+09,13256000.0


In [25]:
#########################book to market ratio
book_to_market = book_value.div(market_caps_df)

display(book_to_market)
save_path_ratio='Financial_ratios/'
book_to_market.to_csv(save_path_ratio + 'book_to_market.csv.gz')

,A,AA,AAAU,AAC,AACC,AACI,AADI,AAI,AAIC,AAL,...,ZVIA,ZVO,ZVXI,ZWRK,ZWS,ZY,ZYME,ZYNE,ZYXI,ZZ
date,,,,,,,,,,,,,,,,,,,,,
2007-01,0.280545,NaN,NaN,NaN,0.446007,NaN,NaN,0.376289,0.924466,NaN,...,NaN,NaN,0.281065,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,0.282754,NaN,NaN,NaN,0.500533,NaN,NaN,0.400531,1.071726,NaN,...,NaN,NaN,0.280192,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,0.268253,NaN,NaN,NaN,0.477237,NaN,NaN,0.404357,1.313906,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,0.262946,NaN,NaN,NaN,0.400807,NaN,NaN,0.376627,1.306803,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,0.236768,NaN,NaN,NaN,0.384524,NaN,NaN,0.334576,1.158613,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,0.193257,1.109260,NaN,1.937153,NaN,NaN,1.474328,NaN,1.518525,NaN,...,1.321657,NaN,NaN,NaN,0.352913,NaN,1.033842,NaN,0.220409,NaN
2023-11,0.156314,1.058877,NaN,NaN,NaN,NaN,1.227672,NaN,1.389086,NaN,...,1.192565,NaN,NaN,NaN,0.317441,NaN,0.793915,NaN,0.213672,NaN
2023-12,0.143484,0.700555,NaN,NaN,NaN,NaN,2.123328,NaN,NaN,NaN,...,0.892390,NaN,NaN,NaN,0.315365,NaN,0.677770,NaN,0.124938,NaN


In [26]:
#########################firm age
firm_age = temp_book_value.dropna(axis = 1, how = 'all').copy()
first_date = temp_book_value.apply(pd.Series.first_valid_index)
last_date = temp_book_value.apply(pd.Series.last_valid_index)
for ticker in firm_age.columns:
    ages = np.arange(1,len(firm_age.loc[first_date[ticker]:last_date[ticker],ticker])+1)
    firm_age.loc[first_date[ticker]:last_date[ticker],ticker] = ages/12

In [27]:
display(firm_age)
save_path_ratio='Financial_ratios/'
firm_age.to_csv(save_path_ratio + 'firm_age.csv.gz')

ticker,BAC,CMA,NCC,PNC,WFC,JPM,CP,ENB,IMO,OKE,...,VIRX,WALD,WEST,WULF,ZFOX,ME,ROIV,BAER,BRCC,OPAL
date,,,,,,,,,,,,,,,,,,,,,
1958-12,0.083333,0.083333,0.083333,0.083333,0.083333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959-12,0.166667,0.166667,0.166667,0.166667,0.166667,0.083333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1960-12,0.250000,0.250000,0.250000,0.250000,0.250000,0.166667,0.083333,0.083333,0.083333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961-08,0.333333,0.333333,0.333333,0.333333,0.333333,0.250000,0.166667,0.166667,0.166667,0.083333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961-09,0.416667,0.416667,0.416667,0.416667,0.416667,0.333333,0.250000,0.250000,0.250000,0.166667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,61.833333,61.833333,61.833333,61.833333,61.833333,61.750000,61.666667,61.666667,61.666667,61.583333,...,0.916667,0.916667,0.916667,0.916667,0.833333,0.666667,0.666667,NaN,NaN,NaN
2023-11,61.916667,61.916667,61.916667,61.916667,61.916667,61.833333,61.750000,61.750000,61.750000,61.666667,...,1.000000,1.000000,1.000000,1.000000,0.916667,0.750000,0.750000,NaN,NaN,NaN
2023-12,62.000000,62.000000,62.000000,62.000000,62.000000,61.916667,61.833333,61.833333,61.833333,61.750000,...,1.083333,1.083333,1.083333,1.083333,1.000000,0.833333,0.833333,0.083333,0.083333,0.083333


In [28]:
#########################compute yearly Return on assets (ROA) for all companies
#ROA = operating income before depreciation / Total assets
net_income = db.get_table(library = 'comp', table = 'funda', columns = ['datadate', 'tic', 'ni'])
net_income.columns = ['date', 'ticker','net_income']
net_income = net_income[net_income.ticker.isin(stocknames.ticker)].drop_duplicates(subset = ['ticker','date'])
net_income.dropna(how = 'any', inplace = True)
net_income.date = pd.to_datetime(net_income.date)
net_income = net_income.pivot(index = 'date', columns = 'ticker', values = 'net_income')
net_income.index = net_income.index.to_period('M')
net_income = net_income[(net_income.index.year > 2006) & (net_income.index.year < 2025)]

net_income=net_income.astype(float)
#Winsorize by year
net_income = net_income.groupby(net_income.index.year).apply(lambda x: x.stack().clip(lower = x.stack().quantile(0.01), upper = x.stack().quantile(0.99)).unstack())
net_income = net_income.ffill()

total_assets = db.get_table(library = 'comp', table = 'funda', columns = ['datadate', 'tic', 'at'])
total_assets.columns = ['date', 'ticker','total_assets']
total_assets = total_assets[total_assets.ticker.isin(stocknames.ticker)].drop_duplicates(subset = ['ticker','date'])
total_assets.dropna(how = 'any', inplace = True)
total_assets.date = pd.to_datetime(total_assets.date)
total_assets = total_assets.pivot(index = 'date', columns = 'ticker', values = 'total_assets')
total_assets.index = total_assets.index.to_period('M')
total_assets = total_assets.replace(0, np.nan)
total_assets = total_assets[(total_assets.index.year > 2006) & (total_assets.index.year < 2025)]

total_assets=total_assets.astype(float)
#Winsorize by year
total_assets = total_assets.groupby(total_assets.index.year).apply(lambda x: x.stack().clip(lower = x.stack().quantile(0.01), upper = x.stack().quantile(0.99)).unstack())
total_assets = total_assets.ffill()

ROA = net_income.div(total_assets)

ROA=ROA.reset_index(level=0, drop=True)

ROA = ROA.groupby(ROA.index.year).apply(lambda x: x.stack().clip(lower = x.stack().quantile(0.01), upper = x.stack().quantile(0.99)).unstack())

ROA=ROA.reset_index(level=0, drop=True)

net_income=net_income.reset_index(level=0, drop=True)
net_income = net_income.multiply(1000000)

total_assets=total_assets.reset_index(level=0, drop=True)
total_assets = total_assets.multiply(1000000)

In [29]:
display(ROA)
save_path_ratio='Financial_ratios/'
ROA.to_csv(save_path_ratio + 'ROA.csv.gz')

ticker,ADSK,AEO,AGX,ALOT,ALOY,ANF,BAMM,BIG,BKE,BKS,...,RGA,SHO,SUI,UFCS,UHT,WHG,WRB,WTM,WU,ZION
date,,,,,,,,,,,,,,,,,,,,,
2007-01,0.161168,0.194899,-0.000933,0.104464,-0.036797,0.187800,0.062121,0.072097,0.151348,0.047087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,0.161168,0.194899,-0.000933,0.104464,-0.036797,0.187800,0.062121,0.072097,0.151348,0.047087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,0.161168,0.194899,-0.000933,0.104464,-0.036797,0.187800,0.062121,0.072097,0.151348,0.047087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,0.161168,0.194899,-0.000933,0.104464,-0.036797,0.187800,0.062121,0.072097,0.151348,0.047087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,0.161168,0.194899,-0.000933,0.104464,-0.036797,0.187800,0.062121,0.072097,0.151348,0.047087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,0.087201,0.036579,0.067618,0.019115,-0.030878,0.001038,0.012024,-0.057088,0.304002,0.002210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11,0.087201,0.036579,0.067618,0.019115,-0.030878,0.001038,0.012024,-0.057088,0.304002,0.002210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12,0.087201,0.036579,0.067618,0.019115,-0.030878,0.001038,0.012024,-0.057088,0.304002,0.002210,...,0.00924,0.065636,-0.012591,-0.009446,0.025823,0.061353,0.037131,0.060721,0.076353,0.00767


In [30]:
display(net_income)
save_path_ratio='Financial_ratios/'
net_income.to_csv(save_path_ratio + 'net_income.csv.gz')

ticker,ADSK,AEO,AGX,ALOT,ALOY,ANF,BAMM,BIG,BKE,BKS,...,RGA,SHO,SUI,UFCS,UHT,WHG,WRB,WTM,WU,ZION
date,,,,,,,,,,,,,,,,,,,,,
2007-01,289700000.0,387359000.0,-113000.0,6059000.0,-7233000.0,422186000.0,18887000.0,124045000.0,55726000.0,150527000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,289700000.0,387359000.0,-113000.0,6059000.0,-7233000.0,422186000.0,18887000.0,124045000.0,55726000.0,150527000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,289700000.0,387359000.0,-113000.0,6059000.0,-7233000.0,422186000.0,18887000.0,124045000.0,55726000.0,150527000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,289700000.0,387359000.0,-113000.0,6059000.0,-7233000.0,422186000.0,18887000.0,124045000.0,55726000.0,150527000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,289700000.0,387359000.0,-113000.0,6059000.0,-7233000.0,422186000.0,18887000.0,124045000.0,55726000.0,150527000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,823000000.0,125136000.0,33098000.0,2661000.0,-4883000.0,2816000.0,3538000.0,-210708000.0,254626000.0,3769000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11,823000000.0,125136000.0,33098000.0,2661000.0,-4883000.0,2816000.0,3538000.0,-210708000.0,254626000.0,3769000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12,823000000.0,125136000.0,33098000.0,2661000.0,-4883000.0,2816000.0,3538000.0,-210708000.0,254626000.0,3769000.0,...,902000000.0,206708000.0,-213300000.0,-29700000.0,15400000.0,9520000.0,1.381359e+09,509200000.0,626000000.0,680000000.0


In [31]:
display(total_assets)
save_path_ratio='Financial_ratios/'
total_assets.to_csv(save_path_ratio + 'total_assets.csv.gz')
total_assets.to_csv(save_path_ratio + 'firm_size.csv.gz')

ticker,ADSK,AEO,AGX,ALOT,ALOY,ANF,BAMM,BIG,BKE,BKS,...,SKYH,SMR,SNTI,SOND,SOUN,WALD,WEST,WULF,ZFOX,BAER
date,,,,,,,,,,,,,,,,,,,,,
2007-01,1.797500e+09,1.987484e+09,121130000.0,58001000.0,196566000.0,2.248067e+09,304037000.0,1.720526e+09,368198000.0,3.196798e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,1.797500e+09,1.987484e+09,121130000.0,58001000.0,196566000.0,2.248067e+09,304037000.0,1.720526e+09,368198000.0,3.196798e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,1.797500e+09,1.987484e+09,121130000.0,58001000.0,196566000.0,2.248067e+09,304037000.0,1.720526e+09,368198000.0,3.196798e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,1.797500e+09,1.987484e+09,121130000.0,58001000.0,196566000.0,2.248067e+09,304037000.0,1.720526e+09,368198000.0,3.196798e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,1.797500e+09,1.987484e+09,121130000.0,58001000.0,196566000.0,2.248067e+09,304037000.0,1.720526e+09,368198000.0,3.196798e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,9.438000e+09,3.420956e+09,489487000.0,139207000.0,158138000.0,2.713100e+09,294251000.0,3.690931e+09,837579000.0,1.705634e+09,...,331204000.0,348635000.0,180792000.0,1.573612e+09,38251000.0,1.090075e+09,746213000.0,317687000.0,766911000.0,NaN
2023-11,9.438000e+09,3.420956e+09,489487000.0,139207000.0,158138000.0,2.713100e+09,294251000.0,3.690931e+09,837579000.0,1.705634e+09,...,331204000.0,348635000.0,180792000.0,1.573612e+09,38251000.0,1.090075e+09,746213000.0,317687000.0,766911000.0,NaN
2023-12,9.438000e+09,3.420956e+09,489487000.0,139207000.0,158138000.0,2.713100e+09,294251000.0,3.690931e+09,837579000.0,1.705634e+09,...,331204000.0,224858000.0,180792000.0,1.573612e+09,151122000.0,1.090075e+09,917832000.0,317687000.0,766911000.0,273470000.0


In [32]:
#########################Intangible assets
intangibles = db.get_table(library = 'comp', table = 'funda', columns = ['datadate', 'tic', 'intan'])
intangibles.columns = ['date','ticker','intangibles']
intangibles = intangibles[intangibles.ticker.isin(stocknames.ticker)].drop_duplicates(subset = ['ticker','date'])
intangibles.dropna(how = 'any', inplace = True)
intangibles.date = pd.to_datetime(intangibles.date)
intangibles = intangibles.pivot(index = 'date', columns = 'ticker', values = 'intangibles')
intangibles.index = intangibles.index.to_period('M')
intangibles = intangibles[(intangibles.index.year > 2006) & (intangibles.index.year < 2025)]

intangibles=intangibles.astype(float)
#Winsorize by year
intangibles = intangibles.groupby(intangibles.index.year).apply(lambda x: x.stack().clip(lower = x.stack().quantile(0.01), upper = x.stack().quantile(0.99)).unstack())
intangibles = intangibles.ffill()
intangibles = intangibles.multiply(1000000)

intangibles=intangibles.reset_index(level=0, drop=True)


intangibles_to_assets = intangibles.div(total_assets)


In [33]:
display(intangibles)
save_path_ratio='Financial_ratios/'
intangibles.to_csv(save_path_ratio + 'intangibles.csv.gz')

ticker,ADSK,AEO,AGX,ALOT,ALOY,ANF,BAMM,BKE,BKS,BTH,...,VIRX,WALD,WEST,WULF,ZFOX,ME,ROIV,BAER,BRCC,OPAL
date,,,,,,,,,,,,,,,,,,,,,
2007-01,4.066000e+08,9950000.0,36642000.0,2337000.0,126642000.0,0.0,1368000.0,0.0,350859000.0,114032000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,4.066000e+08,9950000.0,36642000.0,2337000.0,126642000.0,0.0,1368000.0,0.0,350859000.0,114032000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,4.066000e+08,9950000.0,36642000.0,2337000.0,126642000.0,0.0,1368000.0,0.0,350859000.0,114032000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,4.066000e+08,9950000.0,36642000.0,2337000.0,126642000.0,0.0,1368000.0,0.0,350859000.0,114032000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,4.066000e+08,9950000.0,36642000.0,2337000.0,126642000.0,0.0,1368000.0,0.0,350859000.0,114032000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,4.032000e+09,359481000.0,30642000.0,35890000.0,62248000.0,26147000.0,1224000.0,0.0,373766000.0,8891000.0,...,0.0,973785000.0,244885000.0,0.0,669305000.0,412925000.0,144881000.0,NaN,NaN,NaN
2023-11,4.032000e+09,359481000.0,30642000.0,35890000.0,62248000.0,26147000.0,1224000.0,0.0,373766000.0,8891000.0,...,0.0,973785000.0,244885000.0,0.0,669305000.0,412925000.0,144881000.0,NaN,NaN,NaN
2023-12,4.032000e+09,359481000.0,30642000.0,35890000.0,62248000.0,26147000.0,1224000.0,0.0,373766000.0,8891000.0,...,0.0,973785000.0,239056000.0,0.0,669305000.0,412925000.0,144881000.0,14893000.0,418000.0,56212000.0


In [34]:
display(intangibles_to_assets)
save_path_ratio='Financial_ratios/'
intangibles_to_assets.to_csv(save_path_ratio + 'intangibles_to_assets.csv.gz')

ticker,A,AA,AAC,AACC,AADI,AAI,AAIC,AAL,AAMC,AAME,...,ZTS,ZUMZ,ZUO,ZVIA,ZWS,ZY,ZYME,ZYNE,ZYXI,ZZ
date,,,,,,,,,,,,,,,,,,,,,
2007-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.077134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.077134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.077134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.077134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.077134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,0.41206,0.011792,0.0,0.033722,0.0,0.009896,0.0,0.095031,0.0,0.006931,...,0.276449,0.094944,0.100497,0.037784,0.623848,0.094132,0.059191,0.0,0.253018,0.360217
2023-11,0.41206,0.011792,0.0,0.033722,0.0,0.009896,0.0,0.095031,0.0,0.006931,...,0.276449,0.094944,0.100497,0.037784,0.623848,0.094132,0.059191,0.0,0.253018,0.360217
2023-12,0.41206,0.012928,0.0,0.033722,0.0,0.009896,0.0,0.097402,0.0,0.006931,...,0.286784,0.094944,0.100497,0.038775,0.655568,0.094132,0.059191,0.0,0.208606,0.360217


In [35]:
#########################Tobin's Q
liabilities = total_assets.sub(book_value)
TobinQ = liabilities.add(market_caps_df).div(total_assets)
#Winsorize by year
TobinQ = TobinQ.groupby(TobinQ.index.year).apply(lambda x: x.stack().clip(lower = x.stack().quantile(0.01), upper = x.stack().quantile(0.99)).unstack())
TobinQ=TobinQ.reset_index(level=0, drop=True)

In [36]:
display(liabilities)
save_path_ratio='Financial_ratios/'
liabilities.to_csv(save_path_ratio + 'liabilities.csv.gz')

ticker,A,AA,AAC,AACC,AADI,AAI,AAIC,AAL,AAMC,AAME,...,ZUMZ,ZUO,ZVIA,ZVXI,ZWS,ZY,ZYME,ZYNE,ZYXI,ZZ
date,,,,,,,,,,,,,,,,,,,,,
2007-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62482000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62482000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62482000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62482000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,62482000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,4.918000e+09,9.680000e+09,61152000.0,274681000.0,25859000.0,1.639990e+09,817799000.0,NaN,NaN,264926000.0,...,340575000.0,571438000.0,-29520000.0,NaN,1.249000e+09,113910000.0,155769000.0,9292000.0,50108000.0,NaN
2023-11,4.918000e+09,9.680000e+09,61152000.0,274681000.0,25859000.0,1.639990e+09,817799000.0,NaN,NaN,264926000.0,...,340575000.0,571438000.0,-29520000.0,NaN,1.249000e+09,113910000.0,155769000.0,9292000.0,50108000.0,NaN
2023-12,4.918000e+09,9.904000e+09,61152000.0,274681000.0,27228000.0,1.639990e+09,817799000.0,NaN,NaN,264926000.0,...,340575000.0,571438000.0,996000.0,NaN,1.064200e+09,113910000.0,155769000.0,9292000.0,90775000.0,NaN


In [37]:
display(TobinQ)
save_path_ratio='Financial_ratios/'
TobinQ.to_csv(save_path_ratio + 'TobinQ.csv.gz')

,ADSK,AEO,ALOT,ALOY,ANF,BAMM,BIG,BKE,BKS,BTH,...,LSDI,BATRK,BGLC,CRH,FTLF,AMAM,BRCC,DBD,SEAT,SOUN
date,,,,,,,,,,,,,,,,,,,,,
2007-01,6.000663,3.916061,1.425855,1.060002,3.497163,1.528976,1.993563,2.858269,1.434553,1.585058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,5.670245,3.766993,1.473789,1.047290,3.442513,1.357155,1.935015,2.931290,1.474161,1.569841,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,5.215211,3.651236,1.521777,0.990738,3.347460,1.262768,2.334033,3.073108,1.445224,1.601290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,5.687969,3.553666,1.588537,1.123088,3.560222,1.397445,2.451852,3.111897,1.451429,1.854674,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,6.225613,3.279864,1.576692,1.044059,3.601486,1.392893,2.406016,3.559317,1.518295,1.924465,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08,5.905465,1.511438,1.183346,NaN,1.735746,NaN,0.842043,2.751892,NaN,NaN,...,1.580964,1.093233,3.700262,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09,5.565047,1.491397,1.060669,NaN,1.790160,NaN,0.833451,2.561737,NaN,NaN,...,1.673038,1.073882,2.008571,1.381182,3.326057,NaN,NaN,NaN,NaN,NaN
2023-10,5.359803,1.541042,1.119054,NaN,1.872755,NaN,0.829101,2.584624,NaN,NaN,...,1.049931,1.065779,1.090420,1.362929,3.976346,4.850826,NaN,NaN,NaN,NaN


In [38]:
#########################Market Beta
betas = db.get_table(library = 'comp', table = 'co_mthly', columns = ['datadate', 'gvkey', 'beta'])
betas.columns = ['date', 'gvkey', 'beta']
betas.date = pd.to_datetime(betas.date)
betas = betas.pivot(index = 'date', columns = 'gvkey', values = 'beta')
betas = betas.loc[:,betas.columns.isin(stocknames.gvkey)]
betas.columns = [stocknames.loc[stocknames.gvkey == b].ticker.values[0] for b in betas.columns]
betas.index = betas.index.to_period('M')
betas = betas[(betas.index.year > 2006) & (betas.index.year < 2025)]

betas=betas.astype(float)
#Winsorize by year
betas = betas.groupby(betas.index.year).apply(lambda x: x.stack().clip(lower = x.stack().quantile(0.01), upper = x.stack().quantile(0.99)).unstack())

betas=betas.reset_index(level=0, drop=True)

In [39]:
display(betas)
save_path_ratio='Financial_ratios/'
betas.to_csv(save_path_ratio + 'betas.csv.gz')

,AIR,AAL,ARXX,PNW,PRG,ABT,ACU,BKTI,AE,AMD,...,BRBR,SSIC,MDV,NVCT,DRCT,SMFL,HTCR,ACLX,AREB,ZIMV
date,,,,,,,,,,,,,,,,,,,,,
2007-01,2.0943,5.0016,1.9773,0.8701,0.3167,0.3728,0.5648,0.8425,1.6460,3.3116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,2.0546,5.0438,1.8453,0.8855,0.3819,0.3548,0.5684,0.8037,1.5795,3.2665,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,1.8920,5.1113,1.7539,0.8491,0.3277,0.3925,0.5941,0.8078,1.6249,3.2675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,2.1638,5.0207,1.9296,0.8487,0.5649,0.4365,0.6761,0.8486,1.6629,3.1844,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,2.1355,4.9942,1.8788,0.8085,0.5347,0.4035,0.6415,0.8385,1.5780,3.1695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,1.6401,1.5793,NaN,0.5094,2.1039,0.6801,0.7939,1.1151,0.8618,1.6561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11,1.6551,1.5716,NaN,0.4920,2.0256,0.6958,0.7964,1.0925,0.7770,1.6765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12,1.6256,1.5642,NaN,0.4833,2.0972,0.7249,0.7956,1.1583,0.7798,1.6965,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# more financial ratios 

# debt_assets: debt to assets
# roe: return on equity
# ps: price to sales                                             NEM
# npm: net profit margin                                         NEM   
# pe_exi: price to earnings
# at_turn: asset turnover
# cash_ratio: cash+short-term inv to current liabilities
# gpm: gross profit margin 
# cfm: cash flow margin                                          NEM
# sale_invcap: sales per dollar of invested capital
# capital_ratio: capitalization ratio
# RD_SALE: R&D expenses as a fraction of sales
# roce: return on capital employedinvc

ratios = db.get_table(library='wrdsapps_finratio', table='firm_ratio',
                      columns = ['gvkey','public_date','debt_assets','roe', 'pe_exi', 'gpm',
                                 'at_turn', 'cash_ratio', 'sale_invcap', 'capital_ratio','RD_SALE','roce'])
#rename the columns
ratios.columns = ['gvkey','date','debt2assets','ROE','price2earnings','profit_margin','asset_turnover',
                  'cash_ratio','sales2invested_cap','capital_ratio','r&d2sales','ROCE']
#set the date
#ratios = ratios.rename(columns = {'public_date':'date'})
ratios.date = pd.to_datetime(ratios.date)
ratios.set_index('date',inplace = True)
ratios = ratios.to_period('M').reset_index()
ratios = ratios[ratios.date.dt.year > 2006]
# limit to stocks from 'stocknames'
ratios = ratios[ratios.gvkey.isin(stocknames.gvkey)]
# add the ticker
ratios = ratios.merge(stocknames[['gvkey','ticker']]).drop('gvkey', axis = 1)
# set the index to be date and ticker
ratios.set_index(['date','ticker'], inplace = True)
ratios = ratios.sort_index()

#Winsorize by year
for year in ratios.index.levels[0].year.unique():
    ratios[ratios.index.get_level_values(0).year == year] = \
                ratios[ratios.index.get_level_values(0).year == year].apply(lambda x: 
                x.clip(lower = x.quantile(0.01), upper = x.quantile(0.99)))




# save the ratios
#save_path = '../data/returns/'
#ratios.to_csv(save_path+'fin_ratios.csv.gz')

In [41]:
def series_to_pd(series):
    df = series.copy().reset_index()

    # Using unstack to pivot the DataFrame
    df_pivoted = df.set_index(['date', 'ticker']).unstack()
    
    # If you want to remove the name of the columns and index, you can do:
    df_pivoted.columns = df_pivoted.columns.droplevel()
    df_pivoted.index.name = None

    return df_pivoted

In [42]:
debt2assets=series_to_pd(ratios['debt2assets'])
ROE=series_to_pd(ratios['ROE'])
price2earnings=series_to_pd(ratios['price2earnings'])
profit_margin=series_to_pd(ratios['profit_margin'])
asset_turnover=series_to_pd(ratios['asset_turnover'])
cash_ratio=series_to_pd(ratios['cash_ratio'])
sales2invested_cap=series_to_pd(ratios['sales2invested_cap'])
capital_ratio=series_to_pd(ratios['capital_ratio'])
RnD2sales=series_to_pd(ratios['r&d2sales'])
ROCE=series_to_pd(ratios['ROCE'])

In [43]:
save_path='Financial_ratios/'

debt2assets.to_csv(save_path + 'debt2assets.csv.gz')
ROE.to_csv(save_path + 'ROE.csv.gz')
price2earnings.to_csv(save_path + 'price2earnings.csv.gz')
profit_margin.to_csv(save_path + 'profit_margin.csv.gz')
asset_turnover.to_csv(save_path + 'asset_turnover.csv.gz')
cash_ratio.to_csv(save_path + 'cash_ratio.csv.gz')
sales2invested_cap.to_csv(save_path + 'sales2invested_cap.csv.gz')
capital_ratio.to_csv(save_path + 'capital_ratio.csv.gz')
RnD2sales.to_csv(save_path + 'RnD2sales.csv.gz')
ROCE.to_csv(save_path + 'ROCE.csv.gz')

### Last minute, additional variables...

In [175]:
humans = db.get_table(library = 'comp', table = 'funda', columns = ['datadate', 'tic', 'emp'])

humans.columns = ['date','ticker','humans']
humans = humans[humans.ticker.isin(stocknames.ticker)].drop_duplicates(subset = ['ticker','date'])
humans.dropna(how = 'any', inplace = True)
humans.date = pd.to_datetime(humans.date)
humans = humans.pivot(index = 'date', columns = 'ticker', values = 'humans')
humans.index = humans.index.to_period('M')
humans = humans[(humans.index.year > 2006) & (humans.index.year < 2025)]

humans=humans.astype(float)

humans = humans.multiply(1000)

In [176]:
humans.interpolate(method='linear', axis=0, inplace=True)

In [177]:
humans=humans.astype(float)

In [178]:
humans.columns.name = None

In [179]:
humans

,A,AA,AAC,AACC,AADI,AAI,AAIC,AAL,AAMC,AAME,...,ZUMZ,ZUO,ZVIA,ZVXI,ZWS,ZY,ZYME,ZYNE,ZYXI,ZZ
date,,,,,,,,,,,,,,,,,,,,,
2007-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3030.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3055.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3079.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,18100.0,13475.000000,4.0,906.0,86.500000,8330.0,9.0,131500.0,55.0,142.0,...,9400.0,1549.0,116.375000,NaN,2475.0,507.0,281.250000,27.0,900.0,4276.0
2023-10,18100.0,13516.666667,4.0,906.0,87.333333,8330.0,9.0,131700.0,55.0,142.0,...,9400.0,1549.0,115.916667,NaN,2450.0,507.0,279.833333,27.0,900.0,4276.0
2023-11,18100.0,13558.333333,4.0,906.0,88.166667,8330.0,9.0,131900.0,55.0,142.0,...,9400.0,1549.0,115.458333,NaN,2425.0,507.0,278.416667,27.0,900.0,4276.0


In [180]:
#re-open relevant dataframe
total_assets = pd.read_csv('Financial_ratios/total_assets.csv.gz', compression='gzip', index_col=0)
market_caps_df = pd.read_csv('Financial_ratios/market_caps.csv.gz', compression='gzip', index_col=0)

# Convert index to datetime format and then format it to year-month
total_assets.index = pd.to_datetime(total_assets.index).strftime('%Y-%m')
market_caps_df.index = pd.to_datetime(market_caps_df.index).strftime('%Y-%m')

total_assets.index = pd.PeriodIndex(total_assets.index, freq='M')
market_caps_df.index = pd.PeriodIndex(market_caps_df.index, freq='M')

total_assets=total_assets.sort_index(axis=1)
market_caps_df=market_caps_df.sort_index(axis=1)

In [181]:
market_caps_df

,A,AA,AAAU,AAC,AACC,AACI,AADI,AAI,AAIC,AAL,...,ZVIA,ZVO,ZVXI,ZWRK,ZWS,ZY,ZYME,ZYNE,ZYXI,ZZ
date,,,,,,,,,,,,,,,,,,,,,
2007-01,1.300326e+10,NaN,NaN,NaN,574381750.0,NaN,NaN,1.007957e+09,1.266725e+09,7.944113e+09,...,NaN,NaN,72648720.0,NaN,NaN,NaN,NaN,NaN,NaN,1.365655e+09
2007-02,1.290168e+10,NaN,NaN,NaN,511810250.0,NaN,NaN,9.469512e+08,1.092672e+09,8.175656e+09,...,NaN,NaN,72875040.0,NaN,NaN,NaN,NaN,NaN,NaN,1.578755e+09
2007-03,1.359910e+10,NaN,NaN,NaN,536793530.0,NaN,NaN,9.379899e+08,8.912702e+08,7.304833e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.599979e+09
2007-04,1.387359e+10,NaN,NaN,NaN,639155580.0,NaN,NaN,1.007052e+09,8.961141e+08,6.276967e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.559171e+09
2007-05,1.540747e+10,NaN,NaN,NaN,666220800.0,NaN,NaN,1.133623e+09,1.010730e+09,6.820698e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.513448e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08,3.576117e+10,5.367776e+09,564253750.0,487506760.0,NaN,85856355.0,154476000.0,NaN,1.352736e+08,9.624022e+09,...,128751860.0,NaN,NaN,NaN,5.122720e+09,NaN,491067480.0,71738870.0,277307800.0,NaN
2023-09,3.271708e+10,5.185757e+09,523116700.0,490699040.0,NaN,86381100.0,118705840.0,NaN,1.253184e+08,8.371386e+09,...,111217560.0,NaN,NaN,NaN,4.846003e+09,NaN,430631820.0,68772225.0,273768000.0,NaN
2023-10,3.024472e+10,4.576022e+09,545113800.0,491611120.0,NaN,86381100.0,107423880.0,NaN,1.215120e+08,7.286982e+09,...,97190120.0,NaN,NaN,NaN,4.576204e+09,NaN,476819460.0,NaN,301067520.0,NaN


In [182]:
humans.index

PeriodIndex(['2007-01', '2007-02', '2007-03', '2007-04', '2007-05', '2007-06',
             '2007-07', '2007-08', '2007-09', '2007-10',
             ...
             '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09',
             '2023-10', '2023-11', '2023-12', '2024-01'],
            dtype='period[M]', name='date', length=205)

In [183]:
market_caps_df.index

PeriodIndex(['2007-01', '2007-02', '2007-03', '2007-04', '2007-05', '2007-06',
             '2007-07', '2007-08', '2007-09', '2007-10',
             ...
             '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '2023-08',
             '2023-09', '2023-10', '2023-11', '2023-12'],
            dtype='period[M]', name='date', length=204)

In [184]:
humans_per_capita = humans.div(market_caps_df)
humans_per_tot_assets = humans.div(total_assets)

In [185]:
display(humans_per_capita)
save_path_ratio='Financial_ratios/'
humans_per_capita.to_csv(save_path_ratio + 'humans_per_capita.csv.gz')

,A,AA,AAAU,AAC,AACC,AACI,AADI,AAI,AAIC,AAL,...,ZVIA,ZVO,ZVXI,ZWRK,ZWS,ZY,ZYME,ZYNE,ZYXI,ZZ
date,,,,,,,,,,,,,,,,,,,,,
2007-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,5.532279e-07,0.000003,NaN,8.151636e-09,NaN,NaN,7.286920e-07,NaN,7.181707e-08,0.000016,...,0.000001,NaN,NaN,NaN,5.107302e-07,NaN,6.531101e-07,3.926004e-07,0.000003,NaN
2023-10,5.984516e-07,0.000003,NaN,8.136512e-09,NaN,NaN,8.129788e-07,NaN,7.406676e-08,0.000018,...,0.000001,NaN,NaN,NaN,5.353782e-07,NaN,5.868748e-07,NaN,0.000003,NaN
2023-11,4.840528e-07,0.000003,NaN,NaN,NaN,NaN,6.834267e-07,NaN,6.775329e-08,0.000016,...,0.000001,NaN,NaN,NaN,4.766532e-07,NaN,4.483954e-07,NaN,0.000003,NaN


In [186]:
display(humans_per_tot_assets)
save_path_ratio='Financial_ratios/'
humans_per_tot_assets.to_csv(save_path_ratio + 'humans_per_tot_assets.csv.gz')

,A,AA,AAC,AACC,AADI,AAI,AAIC,AAL,AAMC,AAME,...,ZUMZ,ZUO,ZVIA,ZVXI,ZWS,ZY,ZYME,ZYNE,ZYXI,ZZ
date,,,,,,,,,,,,,,,,,,,,,
2007-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10,0.000002,9.160116e-07,3.946805e-09,0.000002,4.740271e-07,0.000004,8.979186e-09,0.000002,4.675002e-07,3.868535e-07,...,0.000013,0.000002,0.000001,NaN,8.554469e-07,9.705281e-07,4.313589e-07,4.863025e-07,0.000008,0.000004
2023-11,0.000002,9.188353e-07,3.946805e-09,0.000002,4.785503e-07,0.000004,8.979186e-09,0.000002,4.675002e-07,3.868535e-07,...,0.000013,0.000002,0.000001,NaN,8.467179e-07,9.705281e-07,4.291752e-07,4.863025e-07,0.000008,0.000004
2023-12,0.000002,9.607912e-07,3.946805e-09,0.000002,6.720887e-07,0.000004,8.979186e-09,0.000002,4.675002e-07,3.868535e-07,...,0.000013,0.000002,0.000001,NaN,8.998875e-07,9.705281e-07,4.269914e-07,4.863025e-07,0.000007,0.000004


In [187]:
display(humans)
save_path_ratio='Financial_ratios/'
humans.to_csv(save_path_ratio + 'humans.csv.gz')

,A,AA,AAC,AACC,AADI,AAI,AAIC,AAL,AAMC,AAME,...,ZUMZ,ZUO,ZVIA,ZVXI,ZWS,ZY,ZYME,ZYNE,ZYXI,ZZ
date,,,,,,,,,,,,,,,,,,,,,
2007-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3030.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3055.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3079.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,18100.0,13475.000000,4.0,906.0,86.500000,8330.0,9.0,131500.0,55.0,142.0,...,9400.0,1549.0,116.375000,NaN,2475.0,507.0,281.250000,27.0,900.0,4276.0
2023-10,18100.0,13516.666667,4.0,906.0,87.333333,8330.0,9.0,131700.0,55.0,142.0,...,9400.0,1549.0,115.916667,NaN,2450.0,507.0,279.833333,27.0,900.0,4276.0
2023-11,18100.0,13558.333333,4.0,906.0,88.166667,8330.0,9.0,131900.0,55.0,142.0,...,9400.0,1549.0,115.458333,NaN,2425.0,507.0,278.416667,27.0,900.0,4276.0


In [ ]:
db.close()

### Last minute format conversion ...

In [2]:
directory = 'Financial_values/'
industry_dict={}
FF_dict={}
for ticker in tqdm(os.listdir(directory)):
    try:

        file_path = 'Financial_values/'+ticker+'/industry_codes.csv'
    
        industry_table = pd.read_csv(file_path)
        try:
            industry_code=np.bincount(industry_table["industry_codes"]).argmax()
            FF_code=np.bincount(industry_table["FF_Industry_codes"]).argmax()
        except ValueError:
            print("nan")
            industry_code=np.nan
            FF_code=np.nan
    
        industry_dict[ticker]=industry_code
        FF_dict[ticker]=FF_code

    except:
        print(ticker)
        

  0%|          | 0/7083 [00:00<?, ?it/s]

C:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\series.py:1021: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


nan


C:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\series.py:1021: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)
C:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\series.py:1021: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)
C:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\series.py:1021: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


nan
nan
nan


...

nan


C:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\series.py:1021: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)
C:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\series.py:1021: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


nan
nan


C:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\series.py:1021: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)
C:\Users\natha\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\series.py:1021: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


nan
nan


In [3]:
industry_series = pd.Series(industry_dict)
FF_series = pd.Series(FF_dict)

In [7]:
industry_series.to_csv('Financial_ratios/industry_code.csv')
FF_series.to_csv('Financial_ratios/FF_industry_code.csv')